In [1]:
import os

In [2]:
%pwd

'/Users/ravina/Desktop/TextSummarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/ravina/Desktop/TextSummarizer'

In [5]:
#entity: return type of our function
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml,create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.get('data_ingestion')
        if config is None:
            raise ValueError("Data Ingestion configuration is missing or invalid.")

        create_directories([config.get('root_dir')])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
# gor component updation
import os 
import zipfile
import urllib.request as request
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(url=self.config.source_URL,
                                                  filename=self.config.local_data_file
                                                  )
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") #to see the size of the file


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
#pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-25 11:52:56,053: INFO: common: yaml file:config/config.yaml loaded successfully]
[2024-02-25 11:52:56,055: INFO: common: yaml file:params.yaml loaded successfully]
[2024-02-25 11:52:56,056: INFO: common: created directory at:artifacts]
[2024-02-25 11:52:56,057: INFO: common: created directory at:artifacts/data_ingestion]
[2024-02-25 11:53:00,284: INFO: 742848908: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 8377572
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3aea79eaa36efc9339c42d3a61413f75f62f7e5676002d7473efbe842605ace6"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6846:C8BA3:A05A20:A89C70:65DB0DF7
Accept-Ranges: bytes
Date: Sun, 25 Feb 2024 09:52:57 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230130-FRA
X-Cache: 